<h1 style="color: #f73779	;"> Estimation_of_Obesity_Levels </h1>

<h1 style="color: #F08080	;">00. Libraries </h1>

In [ ]:
# 📚 Basic Libraries
import pandas as pd
import numpy as np
import warnings # nobody likes warnings
import scipy.stats as st
import statsmodels.api as sm
import statsmodels.formula.api as smf
from scipy.stats import ttest_ind

# 📊 Visualizations
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

# 🤖 Machine Learning
from sklearn.model_selection import train_test_split
from sklearn.metrics import cohen_kappa_score
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression #esta es nueva
from sklearn.metrics import roc_curve, confusion_matrix, ConfusionMatrixDisplay #esta es nueva, me da el mapita de calor de roc curve
from sklearn.metrics import classification_report #esta es nueva, esta me va a decir todo el reporte
from sklearn.preprocessing import StandardScaler
from lazypredict.Supervised import LazyClassifier, LazyRegressor
from sklearn.model_selection import train_test_split
from sklearn import datasets

In [ ]:
# ⚙️ Settings
pd.set_option('display.max_columns', None) # display all columns
warnings.filterwarnings('ignore') # ignore warnings

<h1 style="color: #F08080;">01. Data Documentation </h1>

| **Variable Name**               | **Role**   | **Type**       | **Demographic** | **Description**                                                                     | **Units** | **Missing Values** |
|----------------------------------|------------|----------------|------------------|-------------------------------------------------------------------------------------|-----------|---------------------|
| `Gender`                        | Feature    | Categorical    | Gender           |                                                                                     |           | no                  |
| `Age`                           | Feature    | Continuous     | Age              |                                                                                     |           | no                  |
| `Height`                        | Feature    | Continuous     |                  |                                                                                     |           | no                  |
| `Weight`                        | Feature    | Continuous     |                  |                                                                                     |           | no                  |
| `family_history_with_overweight`| Feature    | Binary         |                  | Has a family member suffered or suffers from overweight?                           |           | no                  |
| `FAVC`                          | Feature    | Binary         |                  | Do you eat high caloric food frequently?                                           |           | no                  |
| `FCVC`                          | Feature    | Integer        |                  | Do you usually eat vegetables in your meals?                                       |           | no                  |
| `NCP`                           | Feature    | Continuous     |                  | How many main meals do you have daily?                                             |           | no                  |
| `CAEC`                          | Feature    | Categorical    |                  | Do you eat any food between meals?                                                 |           | no                  |
| `SMOKE`                         | Feature    | Binary         |                  | Do you smoke?                                                                      |           | no                  |
| `CH2O`                          | Feature    | Continuous     |                  | How much water do you drink daily?                                                 |           | no                  |
| `SCC`                           | Feature    | Binary         |                  | Do you monitor the calories you eat daily?                                         |           | no                  |
| `FAF`                           | Feature    | Continuous     |                  | How often do you have physical activity?                                           |           | no                  |
| `TUE`                           | Feature    | Integer        |                  | How much time do you use technological devices such as cell phone, videogames, etc.|           | no                  |
| `CALC`                          | Feature    | Categorical    |                  | How often do you drink alcohol?                                                    |           | no                  |
| `MTRANS`                        | Feature    | Categorical    |                  | Which transportation do you usually use?                                           |           | no                  |
| `NObeyesdad`                    | Target     | Categorical    |                  | Obesity level                                                                      |           | no                  |



The target variable is `NObeyesdad`.


<h1 style="color: #F08080;">02. Data Extraction </h1>

In [ ]:
data = pd.read_csv("C:/Users/astri/Ironhack/second_proyect/ObesityDataSet_raw_and_data_sinthetic.csv")


In [ ]:
data

In [ ]:
data.columns

<h2 style="color: #FFA07A;">Columns standardization</h2>

In [ ]:
data.columns = [column.lower().replace(" ", "_") for column in data.columns]

In [ ]:
data.columns = data.columns.str.lower().str.replace(" ", "_")

<h2 style="color: #FFA07A;">Exploring the Data </h2>

In [ ]:
data.shape


In [ ]:
data.info()

<h2 style="color: #FFA07A;">Copy as best practices</h2>

In [ ]:
data.age.nlargest(5)

In [ ]:
data.age.nsmallest(5)

In [ ]:
df = data.copy()

<h1 style="color: #F08080;">03. Data Cleaning </h1>

<h2 style="color: #FFA07A;">Dealing with Duplicates, Nan and Empty Spaces</h2>

In [ ]:
#Duplicated data
df.duplicated().sum()

In [ ]:
#Nan info
df.isna().sum()

In [ ]:
#Empty spaces
df.eq(' ').sum()

<h1 style="color: #F08080;">04. EDA </h1>

<h2 style="color: #FFA07A;">Checking unique values</h2>

In [ ]:
df.nunique().sort_values(ascending=False)

<h2 style="color: #FFA07A;">Divide the Numerical from the Categorical features </h2>

In [ ]:
#We analyze whether any numerics should be categorical.
cat = df.select_dtypes(exclude="number")
num = df.select_dtypes(include="number")

In [ ]:
cat.columns

In [ ]:
num.columns

<h2 style="color: #FFA07A;">Univarible  </h2>


<h3 style="color: #f3b5b5;">Categorical</h3>

Lets remember that for the categorical variables we can do the following:

Frequency tables. Counts and proportions. we can use value_counts() and 
pd.crosstab().


Visualizations: Bar charts, pie charts

In [ ]:
#We want to have only 4: Normal_weight = 0, Overweight = 1, obesity =2, insufficient =3.
mapping = {"Normal_Weight": 0, 
         "Overweight_Level_I": 1,
         "Overweight_Level_II": 1,
         "Obesity_Type_I": 2,
         "Obesity_Type_II" : 2,
         "Obesity_Type_III" : 2,
         "Insufficient_Weight" : 3}
cat['nobeyesdad'].replace(mapping, inplace=True)
frequency_table = cat.nobeyesdad.value_counts()
print(frequency_table)

sns.countplot(
    x='nobeyesdad',
    data=cat,                       
    palette=sns.color_palette('pastel'),  
)
plt.ylabel('Count')
plt.xlabel('Obesity Levels')  
plt.title('Distribution of Obesity Levels') 
plt.show()

In [ ]:
caec_mapping = {
    'Sometimes': 1,
    'Frequently': 1,
    'Always' : 2,
    'no' : 0}
cat['caec'].replace(caec_mapping, inplace=True)
frequency_table= cat.caec.value_counts()
print(frequency_table)

sns.countplot(
    x='caec',
    data=cat,                       
    palette=sns.color_palette('pastel'),  
)
plt.ylabel('Count')
plt.xlabel('Do you eat any food between meals?')  
plt.show()

In [ ]:
#We are going to create a mapping for the calc column in order to have: no = 0, Sometimes = 1 and Always= 2.
calc_mapping = {
    'Sometimes': 1,
    'Frequently': 1,
    'Always' : 2,
    'no' : 0}
cat['calc'].replace(calc_mapping, inplace=True)
frequency_table= cat.calc.value_counts()
print(frequency_table)

sns.countplot(
    x='calc',
    data=cat,                       
    palette=sns.color_palette('pastel'),  
)
plt.ylabel('Count')
plt.xlabel('How often do you drink alcohol?')  
plt.show()

In [ ]:
mtrans_mapping = {
    'Public_Transportation': 0,
    'Automobile': 0,
    'Motorbike' : 0,
    'Walking' : 1,
    'Bike' : 2}
cat['mtrans'].replace(mtrans_mapping, inplace=True)
frequency_table= cat.mtrans.value_counts()
print(frequency_table)

sns.countplot(
    x='mtrans',
    data=cat,                       
    palette=sns.color_palette('pastel'),  
)
plt.ylabel('Count')
plt.xlabel('Which transportation do you usually use?')  
plt.show()

In [ ]:
gender_mapping = {
    'Female': 0,
    'Male': 1}
cat['gender'].replace(gender_mapping, inplace=True)
frequency_table= cat.gender.value_counts()
print(frequency_table)

sns.countplot(
    x='gender',
    data=cat,                       
    palette=sns.color_palette('pastel'),  
)
plt.ylabel('Count')
plt.xlabel('Gender')  
plt.show()

In [ ]:
family_mapping = {
    'no': 0,
    'yes': 1}
cat['family_history_with_overweight'].replace(family_mapping, inplace=True)
frequency_table= cat.family_history_with_overweight.value_counts()
print(frequency_table)

sns.countplot(
    x='family_history_with_overweight',
    data=cat,                       
    palette=sns.color_palette('pastel'),  
)
plt.ylabel('Count')
plt.xlabel('Family history with overweight?')  
plt.show()


In [ ]:
favc_mapping = {
    'no': 0,
    'yes': 1}
cat['favc'].replace(favc_mapping, inplace=True)
frequency_table= cat.favc.value_counts()
print(frequency_table)

sns.countplot(
    x='favc',
    data=cat,                       
    palette=sns.color_palette('pastel'),  
)
plt.ylabel('Count')
plt.xlabel('Do you eat high caloric food frequently?')  
plt.show()

In [ ]:
smoke_mapping = {
    'no': 0,
    'yes': 1}
cat['smoke'].replace(smoke_mapping, inplace=True)
frequency_table= cat.smoke.value_counts()
print(frequency_table)

sns.countplot(
    x='smoke',
    data=cat,                       
    palette=sns.color_palette('pastel'),  
)
plt.ylabel('Count')
plt.xlabel('Do you smoke?')  
plt.show()

In [ ]:
scc_mapping = {
    'no': 0,
    'yes': 1}
cat['scc'].replace(scc_mapping, inplace=True)
frequency_table= cat.scc.value_counts()
print(frequency_table)

sns.countplot(
    x='smoke',
    data=cat,                       
    palette=sns.color_palette('pastel'),  
)
plt.ylabel('Count')
plt.xlabel('Do you monitor the calories you eat daily?')  
plt.show()

<h3 style="color: #f3b5b5;">Numerical</h3>

<h4 style="color: #f3b5b5;">Describe of the numerical variables</h4>

In [ ]:
num.describe().T

As we can see in the table above, the column weight has the highest mean, with a minimum weight is 39 and a maximum weight is 173.

<h4 style="color: #f3b5b5;">Distribution of the numerical variables</h4>

In [ ]:
color = '#F08080'

# grid size
nrows, ncols = 5, 4  # adjust for your number of features

fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=(20, 16))

axes = axes.flatten()

# Plot each numerical feature
for i, ax in enumerate(axes):
    if i >= len(num.columns):
        ax.set_visible(False)  # hide unesed plots
        continue
    ax.hist(num.iloc[:, i], bins=30, color=color, edgecolor='black')
    ax.set_title(num.columns[i])

plt.tight_layout()
plt.show()

<h4 style="color: #f3b5b5;">Outliers</h4>

In [ ]:
color = '#F08080'

# grid size
nrows, ncols = 5, 4 

fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=(20, 16))

axes = axes.flatten()

for i, ax in enumerate(axes):
    if i >= len(num.columns):  # Asegúrate de usar num en lugar de df
        ax.set_visible(False)
        continue
    ax.boxplot(num.iloc[:, i].dropna(), vert=False, patch_artist=True,  # Cambia df a num aquí
               boxprops=dict(facecolor=color, color='black'), 
               medianprops=dict(color='yellow'), whiskerprops=dict(color='black'), 
               capprops=dict(color='black'), flierprops=dict(marker='o', color='red', markersize=5))
    ax.set_title(num.columns[i], fontsize=10)
    ax.tick_params(axis='x', labelsize=8)

plt.tight_layout()
plt.show()


<h2 style="color: #FFA07A;">Bivariable</h2>

Categorical: gender, caec, calc, mtrans, nobeyesdad.


Numerical Continuous: age, height, ncp, weight, ch2o, faf

<h3 style="color: #f3b5b5;">Categorical VS Continuous</h3>


In [ ]:
#Correlation with our target.
round(num.corrwith(cat["nobeyesdad"]).sort_values(ascending=False),2)

As we can see, weight and vegetable intake at meals is actually correlated with our target, something that makes sense if we are looking at obesity levels.

In [ ]:
#nobeyesdad vs Age
sns.boxplot(data=df, x="nobeyesdad", y="age", palette=sns.color_palette('pastel'))
plt.xticks(rotation = 90)
plt.show()

In [ ]:
#nobeyesdad vs Height
sns.boxplot(data=df, x="nobeyesdad", y="height", palette=sns.color_palette('pastel'))
plt.xticks(rotation = 90)
plt.show()

In [ ]:
#nobeyesdad vs Height
sns.boxplot(data=df, x="nobeyesdad", y="ncp", palette=sns.color_palette('pastel'))
plt.xticks(rotation = 90)
plt.show()

In [ ]:
#nobeyesdad vs weight
sns.barplot(data=df, x="nobeyesdad", y="weight", palette=sns.color_palette('pastel'))
plt.xticks(rotation = 90)
plt.show()

In [ ]:
#nobeyesdad vs ch2o
sns.violinplot(data=df, x="nobeyesdad", y="ch2o", palette=sns.color_palette('pastel'))
plt.xticks(rotation = 90)
plt.show()

In [ ]:
#obeyesdad vs faf
sns.barplot(data=df, x="nobeyesdad", y="faf", palette=sns.color_palette('pastel'))
plt.xticks(rotation = 90)
plt.show()

<h3 style="color: #f3b5b5;">Continuous VS Continuous</h3>

In [ ]:
# Calculating the Spearman correlation coefficient between 'weight' and the numerical columns
spearman_correlations_sorted = num.corrwith(df['weight'], method='spearman').sort_values(ascending=False) # positive ones
spearman_correlations_sorted

The Spearman correlation of (0..46) indicates a  positive monotonic relationship between the two variables. This means that when one variable increases, the other also tends to increase, but not necessarily at a constant rate.

In [ ]:
# Calculating the Pearson correlation coefficient between 'weight' and the numerical columns
pearson_correlations_sorted = num.corrwith(df['weight'], method='pearson').sort_values(ascending=False) # positive ones
pearson_correlations_sorted

The Pearson correlation of (0.46) suggests a  positive linear relationship between the weigh and height. 

In [ ]:
num_corr = round(num.corr(), 2)

mask = np.zeros_like(num_corr)
mask[np.triu_indices_from(mask)] = True # optional, to hide repeat half of the matrix

f, ax = plt.subplots(figsize=(25, 15))
sns.set(font_scale=1.5) # increase font size

ax = sns.heatmap(num_corr, mask=mask, annot=True, annot_kws={"size": 12}, linewidths=.5, cmap="coolwarm", fmt=".2f", ax=ax) # round to 2 decimal places
ax.set_title("Dealing with Multicollinearity", fontsize=20) # add title
plt.show()

In [ ]:
# Plotting scatter plots for each numerical column against 'weight' to visualize their relationships
for col in num.columns:
    plt.figure(figsize=(5, 5))
    plt.title('Scatter plot of SalePrice vs ' + col)
    sns.scatterplot(data=df, x=col, y='weight')
    plt.show()

In [ ]:
import statsmodels.api as sm

# Generating a Q-Q plot for 'weight' to check if its distribution follows a normal distribution
sm.qqplot(df['weight'], line='s');
plt.show()

We can conclude that weight follows a normal distribution.

If we want to eliminate the collinearity, we should remove the column `weight` or `height`. However, in this case, we will keep both columns because the columns with the highest correlation with our target are:

- **weight**: 0.18  
- **fcvc**: 0.16  
- **ncp**: 0.10  
- **height**: 0.09

<h2 style="color: #FFA07A;">Concat of the cat and num df</h2>

In [ ]:
#We reset the index in both df in order to merger them with the column index in common.
num.reset_index(inplace=True)
cat.reset_index(inplace=True)

In [ ]:
df_final = cat.merge(num, on = ["index"],how="inner")
df_final.shape

In [ ]:
#We need to delete the column index for our model.
df_final.drop(columns = "index", inplace =True)

In [ ]:
df_final.info()

In [ ]:
df_final.head()

<h1 style="color: #F08080;">05. Hypotesis Testing </h1>

Somethings to remember:
If p-value < alpha → WE REJECT H0



<h2 style="color: #FFA07A;">Hypothesis 1: The mean number of women with obesity is higher than  the mean number of men with obesity.</h2>


H₀:The mean obesity rate in women is higher than the average obesity of men.

​H₁:The mean obesity rate in women is lower than the average obesity of men.

​


In [ ]:
alpha = 0.05

woman_obesity = df_final[(df_final['gender'] == 0) & (df_final['nobeyesdad'] == 2)]['weight']
man_obesity = df_final[(df_final['gender'] == 1) & (df_final['nobeyesdad'] == 2)]['weight']

t_stat, p_value = ttest_ind(woman_obesity, man_obesity, alternative='less')

print(f"T-statistic: {t_stat}, P-value: {p_value}")


if p_value > alpha:
    print("Fail to Reject the Null Hypothesis")
else:
    print("Reject the Null Hypothesis")

In [ ]:
woman_obesity.mean()

In [ ]:
man_obesity.mean()

<h2 style="color: #FFA07A;">Hypothesis 2: Men uses motor transportations more than women.</h2>


H₀: The average use of car by men is higer than the average use of car by women.

​H₁: The average use of car by men is lesser than the average use of car by women.

In [ ]:
#We use the weight column because it need a numerical column to calculate the mean. 

alpha = 0.05

woman_car = df_final[(df_final['gender'] == 0) & (df_final['mtrans'] == 0)]['weight']
man_car = df_final[(df_final['gender'] == 1) & (df_final['mtrans'] == 0)]['weight']

t_stat, p_value = ttest_ind(man_car, woman_car, alternative='less')

print(f"T-statistic: {t_stat}, P-value: {p_value}")


if p_value > alpha:
    print("Fail to Reject the Null Hypothesis")
else:
    print("Reject the Null Hypothesis")


In [ ]:
woman_car.mean()

In [ ]:
man_car.mean()

<h2 style="color: #FFA07A;">Hypothesis 3: There are more woman with family members that have overweight history than men</h2>


H₀: There are more women than men with a family history of overweight.

​H₁: There are less women than men with a family history of overweight.

In [ ]:
alpha = .05

woman_family = df_final[(df_final['gender'] == 0) & (df_final['family_history_with_overweight'] == 1)]['weight']
man_family = df_final[(df_final['gender'] == 1) & (df_final['family_history_with_overweight'] == 1)]['weight']

t_stat, p_value = ttest_ind(woman_family, man_family, alternative='less')

print(f"T-statistic: {t_stat}, P-value: {p_value}")


if p_value > alpha:
    print("Fail to Reject the Null Hypothesis")
else:
    print("Reject the Null Hypothesis")

In [ ]:
woman_family.mean()

In [ ]:
man_family.mean()